In [1]:
#import numpy as np
#import pandas as pd
#import xml.etree.ElementTree as ET
#import zipfile
import os
import csv
import re
from lxml import etree
import scispacy
import spacy
nlp = spacy.load("en_core_sci_md")
import pandas as pd

In [30]:
def tsv_writer(sent_tag, sent, file_name):
    row = []
    with open(file_name, 'a+', newline='', encoding="utf-8") as f:
        csvwriter = csv.writer(f,delimiter = '\t')
        #row.append(art_name)
        row.append(sent)
        row.append(sent_tag)
        csvwriter.writerow(row)
        return

In [4]:
def file_list():
    with zipfile.ZipFile('C:/Users/Helen/Downloads/allofplos_xml/allofplos_xml.zip','r')as z_file:
        for fileName in z_file.namelist():
            yield(fileName)

In [6]:
# for f_name in f_names:
#     fName = f_name.split('.')
#     if fName[1] =='pmed':
#         with zipfile.ZipFile('C:/Users/Helen/Downloads/allofplos_xml/allofplos_xml.zip','r')as ze_file:
#             ze_file.extract(f_name, path='C:/Users/Helen/python_crap/text_files/xml_files/Pmed')

In [2]:
files = (f for f in os.listdir('C:/Users/Helen/python_crap/text_files/xml_files/Pmed'))

In [3]:
art_titles = []
source = 'C:/Users/Helen/python_crap/text_files/xml_files/Pmed/'
destination = 'C:/Users/Helen/python_crap/text_files/xml_files/Pmed/ResArt/' 
for fileName in files:
    article = source+fileName
    #print(f_name)
    art_tree = etree.parse(article)
    artroot = art_tree.getroot()
    #print(artroot.tag)
    #t = re.compile('trial')
    art_type = artroot.find('./front/article-meta/article-categories/subj-group[@subj-group-type="heading"]/subject')
    #print(art_type.text)
    if art_type.text == 'Research Article':
        art_title =artroot.find('./front/article-meta/title-group/article-title')
        art_title_tuple = (fileName,art_title.text, 0)
        art_titles.append(art_title_tuple)

In [5]:
#print(art_titles)

In [7]:
titles_w_space_rm = []
for title in art_titles:
    if title[1] is not None:
        if '\n' in title[1]:
            try:
                str_title = title[1].strip()
                pr_title = ' '.join(str_title.split())
                #print(pr_title)
                titles_w_space_rm.append((title[0],pr_title,title[2]))
            except:
                print(title[1])
        else:
            titles_w_space_rm.append(title)

In [9]:
file_name = 'C:/Users/Helen/python_crap/text_files/titles.tsv'
for title in titles_w_space_rm:
    tsv_writer(title[0],title[1],file_name)


In [4]:
tr_f_name = 'C:/Users/Helen/python_crap/text_files/xml_files/First_batch50/'+'journal.pmed.1002015.xml'
tree = etree.parse(tr_f_name)
root = tree.getroot()
#print(dir(root))
print(root.tag)
#art_type = root.find('./front/article-meta/article-categories/subj-group[@subj-group-type="heading"]/subject')
art_title = root.find('./front/article-meta/title-group/article-title')
print(art_title.text)

In [ ]:
filename = 'C:/Users/Helen/python_crap/text_files/methods090721.tsv'
for fls in trial_art:
    tr_f_name = 'C:/Users/Helen/python_crap/text_files/xml_files/First_batch50/'+fls[1]
    tree = etree.parse(tr_f_name)
    root = tree.getroot()
    body_elm = root.find('./body/sec[@sec-type="materials|methods"]')
    body_subtree = etree.ElementTree(body_elm) 
    etree.strip_tags(body_subtree, 'xref')
    etree.strip_tags(body_subtree, 'ext-link')
    for el in body_subtree.iter():
        if el.tag == 'p':
            doc = nlp(el.text)
            for sent in doc.sents:
                tsv_writer(sent,filename)

In [ ]:
'./front/article-meta/article-categories/subj-group[@subj-group-type="heading"]/subject'
'./front/article-meta/title-group/article-title'

In [40]:
fileName = 'C:/Users/Helen/python_crap/text_files/xml_files/Pmed/ResArtResArtjournal.pmed.1002785.xml'
tree = etree.parse(fileName)
root = tree.getroot()
p_el = root.find('./body/sec[@sec-type="materials|methods"]')
s_tree = etree.ElementTree(p_el)
for el in s_tree.iter():
    etree.strip_tags(el, 'xref')
    if el.tag == 'p':
#         etree.strip_tags(el, 'italic')
#         etree.strip_tags(el, 'xref')
#         etree.strip_tags(el, 'ext-link')
#         print(el.text)
        pass

In [34]:
cl_names = ['art_name','title', 'value']
resartDf = pd.read_csv('C:/Users/Helen/python_crap/text_files/titles.tsv', delimiter = '\t', names = cl_names)
#resartDf.head()
filt = (resartDf['value'] == 1)
resartDf.loc[filt]
rctDf = resartDf.loc[filt]
#rctDf.head()
#rctDf

In [35]:
initial_tag = 'No tag found'
filename = 'C:/Users/Helen/python_crap/text_files/SCRCT_corpus.tsv'
dir_path = 'C:/Users/Helen/python_crap/text_files/xml_files/Pmed/'
for rct in rctDf['art_name'].iteritems():
    fileName = dir_path+rct[1]
    article_tree = etree.parse(fileName)
    article_root = article_tree.getroot()
    #print(article_root.tag)
    body_section = article_root.find('./body/sec[@sec-type="materials|methods"]')
    body_section_subtree = etree.ElementTree(body_section) 
    for el in body_section_subtree.iter():
        etree.strip_tags(el, 'italic')
        etree.strip_tags(el, 'xref')
        etree.strip_tags(el, 'ext-link')
        etree.strip_tags(el, 'sub')
        if el.tag == 'title':
            s_title = el.text.lower()
            #cur_tag = subtitles.loc[s_title,'class']       
            try:
                cur_tag = subtitles.loc[s_title,'class']
            except:
                cur_tag = initial_tag
        if el.tag == 'p':
            #print(el.tag)
            if el.text != None:
                doc = nlp(el.text)
                for sent in doc.sents:
                    tsv_writer(cur_tag,sent,filename)

In [23]:
extracted_subtitles = []
filename = 'C:/Users/Helen/python_crap/text_files/subtitles.tsv'
dir_path = 'C:/Users/Helen/python_crap/text_files/xml_files/Pmed/'
for rct in rctDf['art_name'].iteritems():
    fileName = dir_path+rct[1]
    article_tree = etree.parse(fileName)
    article_root = article_tree.getroot()
    #print(article_root.tag)
    body_section = article_root.find('./body/sec[@sec-type="materials|methods"]')
    body_section_subtree = etree.ElementTree(body_section) 
    for el in body_section_subtree.iter():
        etree.strip_tags(el, 'italic')
        if el.tag == 'title':
            extracted_subtitles.append(el.text)
            #print(el.text)
            #tsv_writer(rct[1],el.text,filename)

In [24]:
pr_subtitles = []
for subtitle in extracted_subtitles:
    if subtitle is not None:
        if subtitle.lower() not in pr_subtitles:
            pr_subtitles.append(subtitle.lower())
filename = 'C:/Users/Helen/python_crap/text_files/subtitles.tsv'
for subtitle in pr_subtitles:
    tsv_writer(0,subtitle,filename)

In [5]:
filename = 'C:/Users/Helen/python_crap/text_files/subtitles.tsv'
col_names = ['title', 'class']
subtitles = pd.read_csv(filename, delimiter = '\t', names = col_names)
subtitles.head()

,title,class
0,materials and methods,NS
1,study design,SDZ
2,ethical approval,E
3,study area,SDS
4,study endpoints,OE


In [7]:
subtitles.set_index('title',inplace = True)

In [9]:
subtitles.head()

,class
title,
materials and methods,NS
study design,SDZ
ethical approval,E
study area,SDS
study endpoints,OE


In [13]:
#dis_filt = (subtitles['class'].str.contains('SDZ'))
subtitles.loc['study design', 'class']
v = subtitles.loc['study design', 'class']
print(type(v))

<class 'str'>


In [14]:
subtitles.head()

,class
title,
materials and methods,NS
study design,SDZ
ethical approval,E
study area,SDS
study endpoints,OE


In [29]:
initial_tag = 'No tag found'
file_name = 'ResArtResArtjournal.pmed.0010064.xml'
dir_path = 'C:/Users/Helen/python_crap/text_files/xml_files/Pmed/'
full_path = dir_path+file_name
article_tree = etree.parse(full_path)
article_root = article_tree.getroot()
#print(article_root.tag)
body_section = article_root.find('./body/sec[@id="s2"]')
body_section_subtree = etree.ElementTree(body_section)
#print(body_section_subtree)
for el in body_section_subtree.iter():
    etree.strip_tags(el, 'italic')
    etree.strip_tags(el, 'xref')
    etree.strip_tags(el, 'ext-link')
    etree.strip_tags(el, 'sub')
    if el.tag == 'title':
        s_title = el.text.lower()
        #cur_tag = subtitles.loc[s_title,'class']       
        try:
            cur_tag = subtitles.loc[s_title,'class']
        except:
            cur_tag = initial_tag
    if el.tag == 'p':
#         print(el.text)
        doc = nlp(el.text)
        for sent in doc.sents:
            print(cur_tag)
            print(sent)

SP
Between August 2000 and December 2003, we enrolled 42 patients infected with HIV who were older than 18 y and on ART; eligibility criteria included CD4 counts of greater than 400 cells/μl on ART with a nadir of no less than 100 cells/μl, ART-mediated suppression (< 500 copies/ml) for more than 6 mo and less than 50 copies/ml at recruitment on any antiretroviral regimen.
SP
Approval of the study protocol was obtained from the institutional review board (IRB) of the Philadelphia Field Initiating Group for HIV Trials (Philadelphia, Pennsylvania, United States).
SP
Written informed consent was obtained from all patients.
SP
Human experimentation guidelines of the United States Department of Health and Human Services and of the authors' institutions were followed.
SP
The study protocol, including the patient consent form, the CONSORT form, and the IRB approval, can be found in Protocols S1–S4.
No tag found
Forty-two eligible patients from the Jonathan Lax Immune Disorder Clinic in Philad